### 图像轮廓

图像轮廓是具有相同颜色或灰度的连续点的曲线.轮廓在形状分析和物体的检测和识别中很有用。
轮廓的作用:
+ 用于图形分析
+ 物体的识别和检测

注意点:
+ 为了检测的准确性，需要先对图像进行二值化或Canny操作。
+ 画轮廓时会修改输入的图像,如果之后想继续使用原始图像，应该将原始图像储存到其他变量中。

### 查找轮廓
+ cv2.findContours(image, mode, method[, contours[, hierarchy[, offset]]])
    + mode: 查找轮廓的模式
        + RETR_EXTERNAL = 0: 只检测外围轮廓
        + RETR_LIST = 1 : 检测的轮廓不建立等级关系，即检测所有轮廓，比较常用
        + RETR_CCOMP=2: 每层最多两级，从小到大，从里到外
        + RETR_TREE=3: 按照树形存储轮廓，从大到小，从左到右

method轮廓近似方法也叫ApproximationMode
+ CHAIN_APPROX_NONE: 保存所有轮廓的点（比较偏大）
+ CHAIN_APPROX_SIMPLE: 只保存角点，比如四边形的4个角，存储信息少，比较常用

返回contours和hierarchy即轮廓和层级

import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('contours.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 二值化
thresh, binary = cv2.threshold(gray,150,255,cv2.THRESH_BINARY)
# 新版本返回两个结果，旧版本返回三个结果
contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
print(type(contours), type(hierarchy))
print(len(contours), hierarchy.shape) # 11 个轮廓, 11 行层级

### 绘制轮廓
+ cv2.drawContours(binary, contours,contourIdx,color,...)
+ contourIdx要绘制的索引编号，-1表示所有的

img = cv2.imread('contours.png')
img_copy = img.copy()
cv2.drawContours(img_copy,contours,-1,color=(0,0,255), thickness=2)
cv2.imshow('img', img)
# 绘制是直接修改原图的
cv2.imshow('img_copy', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

img = cv2.imread('contours.png')
img_copy = img.copy()
# 只绘制第0个轮廓
cv2.drawContours(img_copy,contours,0,color=(0,0,255), thickness=2)
cv2.imshow('img', img)
# 绘制是直接修改原图的
cv2.imshow('img_copy', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 轮廓的面积与周长
轮廓面积指的是每个轮廓中像素点围成的区域的面积，单位为像素

可以通过轮廓的面积区分物体大小识别不同的物体

在找到轮廓后，可能会有许多细小的轮廓，可以通过轮廓的面积来过滤

+ contourArea(contour)
+ arcLength(curve轮廓, closed是否闭合)

import cv2
import numpy as np

img = cv2.imread('contours.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


# 二值化
thresh, binary = cv2.threshold(gray,150,255,cv2.THRESH_BINARY)

# 新版本返回两个结果，旧版本返回三个结果
contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img,contours,0,color=(0,0,255), thickness=12)
area = cv2.contourArea(contours[0]) # 计算第一个轮廓的周长
perimeter = cv2.arcLength(contours[0], closed=True)
print('第1个轮廓的面积为 %s, 周长为%s' % (area,perimeter))
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 多边形逼近与凸包

findContours找到的轮廓信息contours可能过于复杂不平滑，可以使用approxPolyDP函数对该多边形曲线做适当近似，这就是轮廓的多边形近似

approxPolyDP的DP是Douglas-Peucker算法

DP算法思想就是不断找多边形最远的点 加入形成新多边形，直到最远距离小于指定的精度

+ approxPolyDP(curve要近似逼近的轮廓,epsilon要使用的阈值，closed轮廓是否闭合 )



img = cv2.imread('contours2.png')
img_copy = img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img_copy,contours,-1,(0,0,255),3)

approx = cv2.approxPolyDP(contours[0], 50,True)

# 蓝色是近似的轮廓，阈值越大越粗糙
cv2.drawContours(img_copy,[approx],-1,(255,0,0),3)

cv2.imshow('img ', img)
cv2.imshow('img_copy ', img_copy)

cv2.waitKey(0)
cv2.destroyAllWindows()

### 凸包
逼近多边形是轮廓的高度近似，但是有时候，我们希望使用一个多边形的凸包来简化它。
凸包跟逼近多边形很像，只不过它是物体最外层的凸多边形。凸包指的是完全包含原有轮廓，并且仅由轮廓上的点所构成的多边形。
凸包的每一处都是凸的，
即在凸包内连接任意两点的直线都在凸包的内部。在凸包内，任意连续三个点的内角小于180°。

+ convexHull(points[, hull[, clockwise[, returnPoints]]])
    + points即轮廓
    + colckwise顺时针绘制


img = cv2.imread('contours2.png')
img_copy = img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img_copy,contours,-1,(0,0,255),3)

approx = cv2.convexHull(contours[0], 50,True)

# 蓝色是近似的轮廓，阈值越大越粗糙
cv2.drawContours(img_copy,[approx],-1,(255,0,0),3)

cv2.imshow('img ', img)
cv2.imshow('img_copy ', img_copy)

cv2.waitKey(0)
cv2.destroyAllWindows()
print('凸包面积',cv2.contourArea(approx), '轮廓面积', cv2.contourArea(contours[0]))

### 外接矩形
外接矩形分为最小外接矩形和最大外接矩形.
下图中红色矩形是最小外接矩形，绿色矩形为最大外接矩形.

![外接矩形](外接矩形.png)

img = cv2.imread('contours1.png')
img_copy = img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# 最小外接矩形
# rect是一个旋转的矩形，矩形的起始坐标(x,y)，长宽（w，h），旋转角度
rect = cv2.minAreaRect(contours[1])
print('最小外接矩形',rect)
box = cv2.boxPoints(rect)
print('把旋转坐标的4个点计算出来', box) # 是浮点数，坐标要转为整数
box = np.round(box) # 四舍五入
box = np.int0(box)
cv2.drawContours(img_copy, [box], 0, (255, 0, 0))
# cv2.drawContours(img_copy,contours,1,(0,0,255),3)


cv2.imshow('img ', img)
cv2.imshow('img_copy ', img_copy)

cv2.waitKey(0)
cv2.destroyAllWindows()

# 最大外接矩形
img = cv2.imread('contours1.png')
img_copy = img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# 最大外接矩形，可以直接画

x, y, w, h = cv2.boundingRect(contours[1])
print('最大外接矩形x,y,w,h',rect)
cv2.rectangle(img_copy, (x,y), (x+w, y+h), (0,0,255), 3)


cv2.imshow('img_copy ', img_copy)

cv2.waitKey(0)
cv2.destroyAllWindows()

### 图像金字塔
图像金字塔是图像中多尺度表达的一种，
最主要用于图像的分割，
是一种以多分辨率来解释图像的有效但概念简单的结构。
简单来说,图像金字塔是同一图像不同分辨率的子图集合.

图像金字塔最初用于机器视觉和图像压缩，
一幅图像的金字塔是一系列以金字塔形状排列的分辨率逐步降低，
且来源于同一张原始图的图像集合。
其通过梯次向下采样获得，直到达到某个终止条件才停止采样。
金字塔的底部是待处理图像的高分辨率表示，而顶部是低分辨率的近似。
我们将一层一层的图像比喻成金字塔，层级越高．则图像越小，分辨率越低。



### 常见两类图像金字塔

+ 高斯金字塔(Gaussian pyramid):用来向下/降采样，主要的图像金字塔
+ 拉普拉斯金字塔(Laplacian pyramid):用来从金字塔低层图像重建上层未采样图像，在数字图像处理中也即是预测残差，可以对图像进行最大程度的还原，配合高斯金字塔一起使用。

### 高斯金字塔下采样
通过高斯平滑和亚采样（subsampling）来获得一系列下采样图像

（Gi指图片）
![高斯金字塔](高斯金字塔.png)


![上采样](上采样.png)

In [ ]:
import cv2
import numpy as np

# 下采样-高斯金字塔
img = cv2.imread('cat.jpeg')
new = cv2.pyrDown(img)
cv2.imshow('img ', img)
cv2.imshow('down ', new)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 上采样-高斯金字塔
img = cv2.imread('cat.jpeg')
new = cv2.pyrUp(img)
cv2.imshow('img ', img)
cv2.imshow('up ', new)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 拉普拉斯金字塔（用于图片压缩）

没有现成的API，只有公式
![拉普拉斯金字塔](拉普拉斯金字塔.png)

先缩小，再放大，得到原图的大小，然后使用原图减去这个值得到残差

In [46]:
img = cv2.imread('cat.jpeg')
down = cv2.pyrDown(img)
up = cv2.pyrUp(down)
L = cv2.subtract(img, up)
cv2.imshow('L and img',np.hstack((L, img)))
cv2.waitKey(0)
cv2.destroyAllWindows()

# 可以继续对L进行L做down-up操作，得到下一层拉普拉斯金字塔